### This is a framework that will pull Dumbo signal values for payments in SQL table. It will also help in creating rules by iterating over signals and measuring the precision and recall for the rule.

In [1]:
import pandas as pd
import numpy as np

#from pydumbo import merchant_signals as MerchantSignals
from kplib.dumbo import Dumbo, DataframeInput, FeatureOptions,EvaluationOptions, FileInput

from sklearn.metrics import euclidean_distances
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.impute import SimpleImputer
import sklearn.preprocessing
from sklearn.preprocessing import StandardScaler
from scipy.stats import ks_2samp

from pysnowflake.simple import Session
from pysnowflake.utils import retrieve_query
from pysnowflake.utils import write_df_to_snowflake

import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

import json
#import RiskarbiterService
#from pyriskarbiter.services.riskarbiter import RiskarbiterService
#from tqdm import tqdm


from sklearn.model_selection import train_test_split #
from sklearn.tree import DecisionTreeClassifier
pd.options.display.max_columns = 25
pd.options.display.max_rows = 25
pd.set_option('display.float_format', '{:.2f}'.format)

from sklearn import metrics
import blocks
from blocks.filesystem import GCSFileSystem

pd.options.display.max_rows = 4000

/usr/local/lib/python3.7/dist-packages/snowflake/connector/options.py:97: UserWarning: You have an incompatible version of 'pyarrow' installed (4.0.1), please install a version that adheres to: 'pyarrow<3.1.0,>=3.0.0; extra == "pandas"'
  _expected_pyarrow_version,


In [ ]:
## pull the data from table and store it in your folder in ds-risk-prod-snowstage
with Session(app_name="APP_RISK") as ss:
    
    df = ss.download(
        table = "APP_RISK.APP_RISK_TEST.invoice_payments_june_july_2021",
        #path="gs://ds-risk-prod-snowstage/nmavani/invoice_0721/data_snowflake/",
        path = ''
    )

In [ ]:
## add the list of signals you want to pull dumbo values for 
signallist = ['total_invoice_payment_success_count_by_email_from_invoice_buyer_email', #repeat emails used
'total_invoice_success_count_by_merchant_from_payment_merchant', # total invoice count
'invoices_sent_via_email_count_by_unit_token_from_payment_merchant',
'lev_distance_between_invoice_payer_and_cardholder_name_by_id_from_payment',
]

In [ ]:
## call the dumbo to get signal values
df_features_new = Dumbo().get_features(
        entity="REGISTER_PAYMENT",
    # pulling the data from stored location
        input_query=FileInput("gs://ds-risk-prod-snowstage/nmavani/invoice_0721/data_snowflake/", token_col="PAYMENT_TOKEN"),
    # storing the dumbo values in gcs
        output_path="gs://ds-risk-prod-snowstage/nmavani/invoice_0721/data_dumbo",
        feature_options=FeatureOptions(
            features=signallist, 
            include_for_rules_only=True,
            include_deprecated=False,

        ),
        evaluation_options=EvaluationOptions(
            acl_reasons=["risk"]),
    )

In [2]:
## pull the dumbo signals in data frame
df_features = blocks.assemble('gs://ds-risk-prod-snowstage/nmavani/invoice_0721/data_dumbo/results')

In [ ]:
df_features.columns

In [ ]:
df_features.shape

In [ ]:
# check for missing values in dumbo signals
print((df_features.isna().sum()*100/df_features['PAYMENT_TOKEN'].count()).sort_values())

In [ ]:
# exploratory analysis for the field values
df_features.describe().T

In [ ]:
## create features
import operator

def features(signal, df, threshold, sign, columnname=None):
    operatorlookup = {
    '+': operator.add,
    '-': operator.sub,
    '*': operator.mul,
    '/': operator.truediv,
    '>': operator.gt,
    '>=': operator.ge,
    '<': operator.lt,
    '<=': operator.le
    }
    op = operatorlookup.get(sign)
    if columnname is not None:
        new_column = columnname
    else:
        new_column = sign + str(threshold) + signal
    df[new_column] = df[signal].apply(lambda x: 1 if op(x, threshold) else 0)


def features_multiple(signal1, signal2, df, threshold, sign,columnname=None):
    operatorlookup = {
    '+': operator.add,
    '-': operator.sub,
    '*': operator.mul,
    '/': operator.truediv,
    '>': operator.gt,
    '>=': operator.ge,
    '<': operator.lt,
    '<=': operator.le
    }
    op = operatorlookup.get(sign)
    
    if columnname is not None:
        new_column = columnname
    else:
        new_column = sign + str(threshold) + signal2
        
    #df[new_column] = df[signal1].apply(lambda x: 1 if op(x, threshold*df[signal2]) else 0)
    df[new_column] = op(df[signal1], threshold*df[signal2])

In [ ]:
# maintaining a copy
df_features_new_way = df_features.copy()

In [ ]:

df_features_new_way['merchant_invoice_declined_1h'] = (df_features_new_way['invoice_declined_count_in_1h_by_merchant_from_payment_merchant'].gt(0) | \
                                                      df_features_new_way['invoice_on_file_declined_count_in_1h_by_merchant_from_payment_merchant'].gt(0))



features('bin_streak_length_by_merchant_from_payment_merchant', df_features_new_way, 2, '>', '>2_bin_streak')
features('auth_amount_streak_length_by_merchant_from_payment_merchant', df_features_new_way, 2, '>', '>2_auth_streak')
features('bin_streak_length_by_merchant_from_payment_merchant', df_features_new_way, 3, '>', '>3_bin_streak')
features('auth_amount_streak_length_by_merchant_from_payment_merchant', df_features_new_way, 3, '>', '>3_auth_streak')


#EMAIL BASED SIGNALS


features('total_invoice_payment_failed_count_by_email_from_invoice_buyer_email', df_features_new_way, 2, '>', 'gt_2_failed_by_email')


#BROWSER FINGER PRINT BASED SIGNALS

features('auth_amount_streak_length_by_browser_fingerprint_from_payment_browser_fingerprint', df_features_new_way, 2, '>', 'auth_streak_by_bfp')

features('bin_streak_length_by_browser_fingerprint_from_payment_browser_fingerprint', df_features_new_way, 2, '>', 'bin_streak_by_bfp')

features('total_declined_count_by_browser_fingerprint_from_payment_browser_fingerprint', df_features_new_way, 2, '>', 'gt_2_declined_by_bfp')

features('total_count_in_10min_by_browser_fingerprint_from_payment_browser_fingerprint', df_features_new_way, 2, '>', 'gt_1_payment_by_bfp')

features('unique_pans_declined_by_browser_fingerprint_from_payment_browser_fingerprint', df_features_new_way, 2, '>', 'gt_1_pan_declined_by_bfp')



features('CHARGEBACKS_USD', df_features_new_way, 0, '>')
#features('SCORE', df_features_new_way, 0.01, '>=', 'model_score')
features('probability_bf_global_invoice_over100', df_features_new_way, 0.9, '>=', 'model_score')

df_features_new_way['suspected'] = df_features_new_way['IS_SUSPECTED'] == True
df_features_new_way['cased'] = df_features_new_way['IS_CASED'] == True
df_features_new_way['AVS_mismatch'] = df_features_new_way.AUTH_AVS_STATUS == 'AVS_REJECTED'


#COMBINED SIGNALS



df_features_new_way['declined_by_bfp_and_email_and_merchant'] = (df_features_new_way.gt_2_declined_by_bfp |  df_features_new_way.gt_2_failed_by_email) & df_features_new_way.merchant_invoice_declined_1h
df_features_new_way['bin_streak_or_auth_streak'] = df_features_new_way['>2_bin_streak'] | df_features_new_way['>2_auth_streak']
df_features_new_way['bin_or_auth_streak_by_bfp'] = (df_features_new_way.bin_streak_by_bfp | df_features_new_way.auth_streak_by_bfp)
df_features_new_way['bin_or_auth_and_decline'] = df_features_new_way['declined_by_bfp_and_email_and_merchant'] & df_features_new_way['bin_or_auth_streak_by_bfp']
df_features_new_way['bin_or_auth_by_instrument_and_decline'] = df_features_new_way['declined_by_bfp_and_email_and_merchant'] & df_features_new_way['bin_streak_or_auth_streak']
df_features_new_way['bin_or_auth_by_bfp_bin_streak_auth_streak_by_payment_and_decline'] = df_features_new_way['declined_by_bfp_and_email_and_merchant'] & df_features_new_way['bin_or_auth_streak_by_bfp'] & df_features_new_way['bin_streak_or_auth_streak']




In [ ]:
##drop redundant columns and group by using chargebacks and casese
df_group =  df_features_new_way.drop(df_features_new_way.iloc[:, 3:65],axis=1).groupby(['>0CHARGEBACKS_USD', 'suspected']).sum().T

In [ ]:
## get recall and precision, total payments suspected and incremental payments suspected
columns= ('recall%', 'precision%', 'total_payments', 'incremental_payments_suspected', 'incremental_chargebacks_suspected')
df_pr_recall = pd.DataFrame(index=df_group.T.columns, columns=columns)

 ##recall  (chargebacks detected of total chargebacks)
df_pr_recall['recall%']=(df_group[1][True]+df_group[1][False])*100/sum(df_features_new_way['IS_CHARGRBACKED']==1)

 ##precision (charegacks detected of all payments flagged)
df_pr_recall['precision%'] = (df_group[1][True]+df_group[1][False])/(df_group[1][True]+df_group[1][False]+df_group[0][True]+df_group[0][False])*100


df_pr_recall['total_payments'] = df_group[1][True]+df_group[1][False]+df_group[0][True]+df_group[0][False]
df_pr_recall['incremental_payments_suspected'] = (df_group[0][False]+df_group[1][False])
df_pr_recall['incremental_chargebacks_suspected'] = df_group[1][False]


df_pr_recall.sort_values(by=['recall%'])
